In [162]:
from game import GlobalFeatureImportance, LocalFeatureImportance, UnsupervisedFeatureImportance
from environment import Environment
from algorithms.SVARM import StratSVARM
from algorithms.CMCS import CMCS, CMCS_Dependent, CMCS_Independent, CMCS_Length, Selective_CMCS
from algorithms.KernelSHAP import KernelSHAP
import pandas as pd
import os
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [163]:
n=14
k=3
budget=1500
rounds=1000
step_interval=50
metric = "ratio"
env = Environment(n=n, budget=budget, metric=metric)

In [164]:
filepath_global = "datasets/Global feature importance/Adult classification random forest.csv"
filepath_unsupervised = "datasets/Unsupervised Feature Importance/vf_Fifa.csv"
directory_local = "datasets/adult_classification"
# game = GlobalFeatureImportance(filepath=filepath_global, num_players=n, use_cached=True)
game = LocalFeatureImportance(directory=directory_local, num_players=n)
# game = UnsupervisedFeatureImportance(filepath=filepath_unsupervised, num_players=n)
# game = SumUnanimityGames()

In [165]:
algorithms = [("StratSVARM(False,False)", StratSVARM(start_exact=False, theoretical_distribution=False)), ("KernelSHAP", KernelSHAP())]

In [166]:
results = []
for name, algorithm in algorithms:
    results += [(name, env.evaluate(game, algorithm, k, step_interval, rounds))]
columns = ["budget", "topk_ratio", "topk_ratio_SE", "mse", "mse_SE", "topk_sum", "topk_sum_SE"]

In [167]:
for algorithm, result in results:
    df = pd.DataFrame(data={column: result[index] for index, column in enumerate(columns)})
    dir = f"results/data/{game.__class__.__name__}/{game.name}/budget={budget}_rounds={rounds}/k={k}"
    if not os.path.isdir(dir):
        os.makedirs(dir)
    df.to_csv(f"{dir}/{algorithm}.csv", index=False)